In [110]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import features
import os
import shapely
import matplotlib.pyplot as plt
from shapely.geometry import Point
import random

In [231]:

input_file = "data/sonneberg_startlastchangeperiod.tif"
absolute_path = os.path.abspath(input_file)
src = rasterio.open(path)
src.crs

CRS.from_epsg(32632)

In [230]:
# reproject file if necessary
if src.crs != 32632:
    absolute_path = absolute_path[:absolute_path.rfind("/")+1] # path of file
    in_file_name = absolute_path[absolute_path.rfind('/')+1:]
    out_file_name = absolute_path[:absolute_path.rfind(".")]+"_32632.tif"
    command = "gdalwarp -t_srs EPSG:32632 " + absolute_path+in_file_name + " " + out_file_name

In [73]:
def to_vector(path):
    # read raster_path
    raster_path = rasterio.open(path)
    raster_array = raster_path.read(1)
    
    # transform to polygons
    tmp_value = []
    tmp_shape = []
    tmp_geom = []
    for geom, value in features.shapes(raster_array, transform=raster_path.transform):
        #print(value)
        tmp_value.append(value)
        #pprint.pprint(geom)
        tmp_shape.append(geom["type"])
        tmp_geom.append(geom["coordinates"])
    
    # turn returned dictionary into WKT string
    tmp_geom2 = []
    for a,b in zip(tmp_shape,tmp_geom):
        tmp_str = str(b[0])
        tmp_str = tmp_str.replace("), ",",")
        tmp_str = tmp_str.replace(", "," ")
        tmp_str = tmp_str.replace("[","")
        tmp_str = tmp_str.replace("]","")
        tmp_str = tmp_str.replace("(","")
        tmp_str = tmp_str.replace(")","")
        tmp_str = (str(a + " " + "((" + tmp_str + "))"))
        tmp_str = tmp_str.upper()
        tmp_geom2.append(shapely.wkt.loads(tmp_str))
        
    gdf = gpd.GeoDataFrame({'value': tmp_value, 'geometry': tmp_geom2 },crs=raster_path.crs)
    gdf = gdf[gdf.value != 0.0] # delete 0 values
    return(gdf) 

In [75]:
# create vector geodataframe from image
gdf = to_vector(path)

In [95]:
# append area of geometry to each row
gdf["area"] = gdf["geometry"].area
# extracting unique values
values = gdf["value"].unique()
print("unique values: ",values)
gdf

unique values:  [31. 34. 18. 38. 40. 35. 32. 26. 33. 37. 39. 30. 28. 24. 36. 27. 20. 22.
 23. 21. 29.  9. 10. 14. 19. 16. 13. 15. 25.  7. 12. 11.  2. 17.  3.  6.
  8.  5.  4.  1.]


,value,geometry,area
0,31.0,"POLYGON ((661585.568 5601638.952, 661585.568 5...",3444.402602
1,34.0,"POLYGON ((661657.447 5601638.952, 661657.447 5...",1148.134201
2,18.0,"POLYGON ((661681.406 5601638.952, 661681.406 5...",15499.811709
3,38.0,"POLYGON ((661130.333 5601614.992, 661130.333 5...",9185.073606
4,40.0,"POLYGON ((661250.132 5601614.992, 661250.132 5...",574.067100
...,...,...,...
68425,38.0,"POLYGON ((640189.558 5570275.708, 640189.558 5...",574.067100
68426,37.0,"POLYGON ((638991.573 5570251.748, 638991.573 5...",574.067100
68427,35.0,"POLYGON ((639015.533 5570251.748, 639015.533 5...",1148.134201
68428,40.0,"POLYGON ((639111.372 5570251.748, 639111.372 5...",574.067100


In [111]:
# create dict of absolute area by value
area_dict = {}
for i in values:
    # append sum of area for each vector value
    area_dict[i] = gdf[gdf.value==i].sum()["area"]

In [138]:
# create dict of area percentages by value
total_area = gdf["area"].sum()
perc_dict = {}
for i in values:
    part = area_dict[i]
    perc_dict[i] = 100 * float(part)/float(total_area)

In [139]:
# how much of total area does each polygon occupy
gdf["area_perc"] = 100 * gdf["area"] / float(total_area)

In [141]:
# function to randomly put set number of points in polygon
def random_points_in_polygon(number, polygon):
    points = []
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(point):
            points.append(point)
            i += 1
    return points  # returns list of shapely point

In [215]:
# define total no of points to be created
num_points = 1000
gdf_points = gpd.GeoDataFrame({'value': [], 'geometry': [] },crs="EPSG:32632")

In [219]:
# get absolute no. of points from avg. no of points per polygon
def no_points_from_avg(num):
    full_points = int(num)
    likelihood_next_point = num-full_points
    if likelihood_next_point > 0.0:
        rand_float = random.random()
        if rand_float<likelihood_next_point:
            full_points = full_points+1
    return full_points


for index, row in gdf.iterrows():
    current_value = row["value"] # get current value
    current_polygon = row["geometry"] # get current polygon
    current_area_perc = row["area_perc"] # get current polygon total area percentage
    value_perc_of_points = perc_dict[current_value] # get total point percentage for current value
    no_of_points_for_current_polygon = num_points/100.0 * current_area_perc # calculate no. of points that should go in this polygon on average
    
    # get no of points weighted for this polygon size
    current_no_of_points = no_points_from_avg(no_of_points_for_current_polygon)
    
    # create list holding values in order to have value value of poit in df
    value_ls = []
    for i in range(0,current_no_of_points):
        value_ls.append(current_value)
    
    # if point applicable for this polygon
    if current_no_of_points != 0:
        # create specified number of random points in polygon
        gdf_temp = gpd.GeoDataFrame({'value': value_ls, 'geometry': random_points_in_polygon(current_no_of_points, current_polygon) },crs="EPSG:32632")
        # append points to global geodataframe
        gdf_points = gdf_points.append(gdf_temp,ignore_index=True)

In [224]:
# append coordinates to gdf
gdf_points['lon'] = round(gdf_points.geometry.x,3)
gdf_points['lat'] = round(gdf_points.geometry.y,3)

In [233]:
gdf_points.to_csv("sonneberg_startlastchangeperiod.csv")